In [ ]:
# 安裝必要套件（首次執行需要）
# !pip install tf2onnx onnx onnxruntime onnx-simplifier

In [1]:
import tensorflow as tf
import tf2onnx
import onnx
import onnxruntime as ort
import numpy as np
from PIL import Image
import os

print(f"TensorFlow version: {tf.__version__}")
print(f"tf2onnx version: {tf2onnx.__version__}")
print(f"ONNX version: {onnx.__version__}")
print(f"ONNX Runtime version: {ort.__version__}")

2025-12-24 00:28:38.243750: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-24 00:28:38.308388: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766507318.362400    6862 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766507318.373203    6862 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-24 00:28:38.461408: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

ModuleNotFoundError: No module named 'tf2onnx'

In [ ]:
# 設定路徑
KERAS_MODEL_PATH = '/mnt/f/course/embedded_system/dataset/RAF-DB_DATASET/model/best_anger_model.keras'
ONNX_OUTPUT_PATH = '/mnt/f/course/embedded_system/dataset/RAF-DB_DATASET/model/anger_model.onnx'
ONNX_SIMPLIFIED_PATH = '/mnt/f/course/embedded_system/dataset/RAF-DB_DATASET/model/anger_model_simplified.onnx'

# 載入 Keras 模型
print(f"載入 Keras 模型: {KERAS_MODEL_PATH}")
model = tf.keras.models.load_model(KERAS_MODEL_PATH)
model.summary()

# 取得輸入形狀
input_shape = model.input.shape
print(f"\n模型輸入形狀: {input_shape}")

In [ ]:
# 方法 1: 使用 tf2onnx.convert 直接轉換（推薦）
print("開始轉換 Keras -> ONNX...")

# 定義輸入規格
input_signature = [tf.TensorSpec(model.input.shape, tf.float32, name='input')]

# 轉換為 ONNX
onnx_model, _ = tf2onnx.convert.from_keras(
    model,
    input_signature=input_signature,
    opset=13,  # ONNX opset 版本，13 對 TensorRT 相容性較好
    output_path=ONNX_OUTPUT_PATH
)

print(f"✅ 轉換完成！模型已儲存至: {ONNX_OUTPUT_PATH}")
print(f"ONNX 模型大小: {os.path.getsize(ONNX_OUTPUT_PATH) / (1024*1024):.2f} MB")

In [ ]:
# 驗證 ONNX 模型
print("驗證 ONNX 模型...")
onnx_model_check = onnx.load(ONNX_OUTPUT_PATH)
onnx.checker.check_model(onnx_model_check)
print("✅ ONNX 模型驗證通過！")

# 顯示模型資訊
print(f"\n模型資訊:")
print(f"  IR 版本: {onnx_model_check.ir_version}")
print(f"  生產者: {onnx_model_check.producer_name}")
print(f"  Opset: {onnx_model_check.opset_import[0].version}")

# 顯示輸入輸出資訊
print(f"\n輸入:")
for input_tensor in onnx_model_check.graph.input:
    print(f"  名稱: {input_tensor.name}")
    print(f"  形狀: {[dim.dim_value for dim in input_tensor.type.tensor_type.shape.dim]}")
    print(f"  資料型別: {input_tensor.type.tensor_type.elem_type}")

print(f"\n輸出:")
for output_tensor in onnx_model_check.graph.output:
    print(f"  名稱: {output_tensor.name}")
    print(f"  形狀: {[dim.dim_value for dim in output_tensor.type.tensor_type.shape.dim]}")
    print(f"  資料型別: {output_tensor.type.tensor_type.elem_type}")